In [1]:
"""
Enhanced Bead Trade Coding Notebook - Complete Version with Smart Session Management
===================================================================================

This version includes:
- All original codebook and processing logic
- Smart session management (resume in same folder)
- Complete structured output with dual columns
- XML to JSON parsing
- Batch processing with proper resume functionality
"""

import subprocess
import sys

# === INSTALL DEPENDENCIES IF NEEDED ===
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

for pkg in ["anthropic", "openpyxl", "xlrd", "pandas"]:
    try:
        __import__(pkg if pkg != "openpyxl" else "openpyxl")
    except ImportError:
        install(pkg)

import pandas as pd
from anthropic import Anthropic
import json
import datetime
import os
import re
import xml.etree.ElementTree as ET
import glob

# === GOOGLE DRIVE MOUNT ===
from google.colab import drive
drive.mount('/content/drive')

print("🔬 STRUCTURED BEAD TRADE CODING - V3 WITH SMART SESSIONS")
print("=" * 70)
print("📅 Current date:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("=" * 70)

# === CONFIGURATION SETTINGS ===
EXCEL_PATH = "All_entries_beads_cleaned.xlsx"
ANTHROPIC_API_KEY = "YOUR_API_KEY_HERE" #update with your API key
MODEL_NAME = "claude-3-5-sonnet-20241022"  # Updated model name ("claude-3-7-sonnet-20250219"; "claude-sonnet-4-20250514")
TEMPERATURE = 0.2
LOG_FILE = "structured_annotation_log.txt"
SAVE_EVERY = 50

# Processing settings
BATCH_SIZE = 2000
START_ROW = 0  # Will be updated based on session choice
END_ROW = None
TEXT_COLUMN = "text_page_gp"

# === COMPREHENSIVE STRUCTURED CODEBOOK (YOUR ORIGINAL) ===
STRUCTURED_CODEBOOK = """
STRUCTURED BEAD TRADE CODING CODEBOOK - EXPERT-VALIDATED VERSION
================================================================
Based on highest human-AI agreement rates (July 2025) with edge case improvements

You are analyzing historical texts about pre-colonial African bead trade.
Extract information for EVERY variable below, providing both coded values and descriptive details.

CRITICAL INSTRUCTIONS:
1. Answer ALL variables in the order listed below
2. Use "none" or "no" if information is not present
3. Base answers ONLY on the text provided (temperature 0.2)
4. Be CONSERVATIVE - require explicit evidence, do NOT infer
5. The decision tree is for thinking only - still answer every variable
6. Provide both coded answer and descriptive details for each variable

### DATA QUALITY PRE-CHECK:
Before coding, check if the text:
- Has <50 readable characters → Mark as "data_quality_issue"
- Is corrupted OCR (mostly symbols/numbers) → Mark as "corrupted_ocr"
- Is a context page without bead content → Mark as "context_page_only"
- Contains no relevant bead information → Mark as "no_bead_content"

### COMPLETE VARIABLE LIST (ALL MUST BE ANSWERED):

1. **4a_exchange**: Were beads exchanged in a specific transaction?
   - CODE VALUES: "no" or "xo"
   - "xo" = ONLY when text explicitly describes beads being traded/bartered/sold for something specific
   - "no" = ALL other mentions including:
     * Manufacturing/processing ("beads melt down", "fashioned from glass")
     * Value descriptions ("worth more than gold", "highly prized")
     * Personal adornment ("wore strings of beads", "around necks")
     * Market displays ("beads displayed in stalls", "exposed for sale")
     * General demand ("beads in great demand", "scarce items")
     * Ceremonial use ("offerings to gods", "ritual objects")
     * Payment systems ("teaspoonful per day", "wages in beads")
     * Historical generalizations ("natives sold ivory for beads" without specific instance)
     * Observational contexts ("showed rosary beads")
   - SPECIAL CASES:
     * Gift-giving with "offered presents" = "xo" (social exchange)
     * Intangible exchanges ("bought secret for beads") = "xo"
     * Equipment beads (saddlery) = "no" (aesthetic function)

2. **4b_beads_exchanged**: What specific beads were exchanged?
   - Extract: material, color, size, quantity, quality, origin
   - If 4a_exchange="no", use "none"

3. **4c_exchanged_item**: What were beads exchanged FOR?
   - What was received in return for beads
   - If 4a_exchange="no", use "none"

4. **5_exchange_rate**: What was the exchange rate?
   - Specific quantities/ratios mentioned
   - Example: "50 strings for one slave"

5. **6_bead_ethnic_group**: Which ethnic groups/traders were associated?
   - List all groups, origins, trader types
   - Include geographic origins

6. **7_direction_of_trade**: What was the trade direction?
   - Example: "coast to interior", "north to south"

7. **8_location_name**: What locations are mentioned?
   - All place names, geographic descriptors

8. **9_place_of_manufacture**: Where were beads manufactured?
   - Manufacturing locations, craft centers

9. **10_beads_observed**: What beads are described (not necessarily exchanged)?
   - ALL bead descriptions regardless of context
   - Include beads that are worn, displayed, manufactured, valued
   - Material, color, size, shape, quality

10. **11_bead_value**: What value statements are made about beads?
    - Price, worth, comparative value
    - Example: "worth more than gold"

11. **12_local_name**: Local/indigenous names for beads?
    - Indigenous terms, meanings

12. **13_notes**: Additional relevant information
    - Context not captured elsewhere

13. **14_trading_partners**: Specific individuals/groups in transactions
    - Names, roles, relationships

14. **15_temporal_context**: Time references
    - Dates, seasons, periods mentioned

OUTPUT FORMAT:
Provide your response in XML format with both coded values and descriptive details:

<analysis>
<data_quality>
[Check: readable/corrupted_ocr/context_page_only/no_bead_content]
</data_quality>

<thinking>
[Your analysis process here - apply conservative decision rules]
</thinking>

<variables>
<var_read_entry>0, 1, 2, or NaN</var_read_entry>
<var_read_entry_detail>Entry readability and bead relevance</var_read_entry_detail>

<var_1a_physical_function>2 or NA</var_1a_physical_function>
<var_1a_physical_function_detail>Aesthetic/jewelry functions described</var_1a_physical_function_detail>

<var_1b_trade_function>2 or NA</var_1b_trade_function>
<var_1b_trade_function_detail>Currency/exchange functions described</var_1b_trade_function_detail>

<var_1c_social_function>3 or NA</var_1c_social_function>
<var_1c_social_function_detail>Ceremonial/social functions described</var_1c_social_function_detail>

<var_2_nature_of_exchange>1, 2, 3, 4, or NA</var_2_nature_of_exchange>
<var_2_nature_of_exchange_detail>Exchange dynamics if applicable</var_2_nature_of_exchange_detail>

<var_3_between_groups>1, 2, 3, 4, or NA</var_3_between_groups>
<var_3_between_groups_detail>Groups involved with names</var_3_between_groups_detail>

<var_4a_exchange>no or xo</var_4a_exchange>
<var_4a_exchange_detail>Explicit exchange evidence</var_4a_exchange_detail>

<var_4b_beads_exchanged>description or none</var_4b_beads_exchanged>
<var_4b_beads_exchanged_detail>Full bead characteristics</var_4b_beads_exchanged_detail>

<var_4c_exchanged_item>item or none</var_4c_exchanged_item>
<var_4c_exchanged_item_detail>What beads were exchanged for</var_4c_exchanged_item_detail>

<var_5a_related_raw_materials>list or NA</var_5a_related_raw_materials>
<var_5a_related_raw_materials_detail>Raw materials context</var_5a_related_raw_materials_detail>

<var_5b_related_jewerly_fashion>list or NA</var_5b_related_jewerly_fashion>
<var_5b_related_jewerly_fashion_detail>Fashion items context</var_5b_related_jewerly_fashion_detail>

<var_5c_related_consumerables>list or NA</var_5c_related_consumerables>
<var_5c_related_consumerables_detail>Consumables context</var_5c_related_consumerables_detail>

<var_5d_related_decoratives>list or NA</var_5d_related_decoratives>
<var_5d_related_decoratives_detail>Decorative items context</var_5d_related_decoratives_detail>

<var_6_bead_ethnic_group>groups or none</var_6_bead_ethnic_group>
<var_6_bead_ethnic_group_detail>Full ethnic context</var_6_bead_ethnic_group_detail>

<var_7_market_town>0 or 1</var_7_market_town>
<var_7_market_town_detail>Market designation</var_7_market_town_detail>

<var_8_location_name>location or none</var_8_location_name>
<var_8_location_name_detail>Complete geographic details</var_8_location_name_detail>

<var_9_place_of_manufacture>place or none</var_9_place_of_manufacture>
<var_9_place_of_manufacture_detail>Manufacturing origins</var_9_place_of_manufacture_detail>

<var_10a_size>size or NA</var_10a_size>
<var_10a_size_detail>Size descriptions</var_10a_size_detail>

<var_10b_color>color or NA</var_10b_color>
<var_10b_color_detail>Color descriptions</var_10b_color_detail>

<var_10c_shape>shape or NA</var_10c_shape>
<var_10c_shape_detail>Shape descriptions</var_10c_shape_detail>

<var_10d_type>type or NA</var_10d_type>
<var_10d_type_detail>Material type descriptions</var_10d_type_detail>

<var_11_units_of_measurement>1, 2, 3, 4, or NA</var_11_units_of_measurement>
<var_11_units_of_measurement_detail>Measurement methods</var_11_units_of_measurement_detail>

<var_12_local_name>name or none</var_12_local_name>
<var_12_local_name_detail>Indigenous terminology</var_12_local_name_detail>

<var_13_notes>notes or none</var_13_notes>
<var_13_notes_detail>Additional historical context</var_13_notes_detail>
</variables>
</analysis>
"""

# === SESSION MANAGEMENT FUNCTIONS ===
def analyze_session(session_path):
    """Analyze a session folder and return its status."""
    session_info = {
        'path': session_path,
        'name': os.path.basename(session_path),
        'last_row': None,
        'batch_start': None,
        'batch_end': None,
        'files_count': 0,
        'total_size_mb': 0,
        'status': 'unknown'
    }

    # Check last index
    index_file = os.path.join(session_path, 'last_index.txt')
    if os.path.exists(index_file):
        with open(index_file, 'r') as f:
            session_info['last_row'] = int(f.read().strip())

    # Check batch info
    batch_file = os.path.join(session_path, 'batch_info.txt')
    if os.path.exists(batch_file):
        with open(batch_file, 'r') as f:
            lines = f.readlines()
            if len(lines) >= 2:
                session_info['batch_start'] = int(lines[0].strip())
                session_info['batch_end'] = int(lines[1].strip())

    # Count Excel files and calculate size
    excel_files = glob.glob(os.path.join(session_path, "*.xlsx"))
    session_info['files_count'] = len(excel_files)

    for file in excel_files:
        session_info['total_size_mb'] += os.path.getsize(file) / (1024 * 1024)

    # Determine status
    if session_info['last_row'] is not None and session_info['batch_end'] is not None:
        if session_info['last_row'] >= session_info['batch_end'] - 1:
            session_info['status'] = 'complete'
        else:
            remaining = session_info['batch_end'] - session_info['last_row'] - 1
            progress = ((session_info['last_row'] - session_info['batch_start'] + 1) /
                       (session_info['batch_end'] - session_info['batch_start'])) * 100
            session_info['status'] = f'{progress:.1f}% ({remaining} rows left)'
    elif session_info['last_row'] is not None:
        session_info['status'] = f'row {session_info["last_row"]} processed'
    else:
        session_info['status'] = 'not started'

    return session_info

def select_or_create_session():
    """
    Main session management function.
    Shows existing sessions and lets user choose to resume or create new.
    Returns: (session_path, start_row, is_new_session)
    """
    base_dir = "/content/drive/MyDrive/bead_annotation"
    os.makedirs(base_dir, exist_ok=True)

    # Find all existing session folders
    pattern = os.path.join(base_dir, "structured_session_*")
    existing_sessions = sorted(glob.glob(pattern), reverse=True)

    if not existing_sessions:
        # No existing sessions - create new
        print("📁 No existing sessions found. Creating new session...")
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        new_path = os.path.join(base_dir, f"structured_session_{timestamp}")
        os.makedirs(new_path, exist_ok=True)
        print(f"✅ Created: {os.path.basename(new_path)}")
        return new_path, 0, True

    # Analyze existing sessions
    print("\n📁 EXISTING SESSIONS:")
    print("=" * 70)

    sessions_data = []
    for i, session_path in enumerate(existing_sessions[:10], 1):  # Show max 10
        info = analyze_session(session_path)
        sessions_data.append(info)

        # Format display
        name = info['name']

        # Extract date from name
        date_match = re.search(r'(\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2})', name)
        if date_match:
            date_str = date_match.group(1).replace('_', ' ')
        else:
            date_str = "unknown date"

        print(f"\n{i}. {name}")
        print(f"   📅 Created: {date_str}")

        if info['batch_start'] is not None and info['batch_end'] is not None:
            print(f"   📊 Batch: rows {info['batch_start']}-{info['batch_end']-1}")

        if info['last_row'] is not None:
            print(f"   ✏️ Last row: {info['last_row']}")

        print(f"   📁 Files: {info['files_count']} ({info['total_size_mb']:.1f} MB)")
        print(f"   📈 Status: {info['status']}")

    print("\n" + "=" * 70)
    print("\nOPTIONS:")
    print("  1-10 : Resume specific session")
    print("  N    : Create NEW session")
    print("  B    : Create new session for specific BATCH number")
    print("  A    : AUTO-select most recent incomplete session")

    while True:
        choice = input("\nYour choice: ").strip().upper()

        if choice == 'A':
            # Find most recent incomplete session
            for info in sessions_data:
                if info['status'] != 'complete' and info['status'] != 'not started':
                    print(f"\n✅ Auto-selected: {info['name']}")
                    print(f"   Status: {info['status']}")

                    if info['last_row'] is not None:
                        start_row = info['last_row'] + 1
                    else:
                        start_row = info['batch_start'] if info['batch_start'] is not None else 0

                    return info['path'], start_row, False

            print("❌ No incomplete sessions found. Creating new...")
            choice = 'N'

        if choice == 'N':
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            new_path = os.path.join(base_dir, f"structured_session_{timestamp}")
            os.makedirs(new_path, exist_ok=True)
            print(f"\n✅ Created new session: {os.path.basename(new_path)}")
            return new_path, 0, True

        elif choice == 'B':
            batch_num = input("Enter batch number (1, 2, 3, etc.): ").strip()
            try:
                batch_num = int(batch_num)
                start_row = (batch_num - 1) * BATCH_SIZE
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                new_path = os.path.join(base_dir, f"structured_session_{timestamp}_batch{batch_num}")
                os.makedirs(new_path, exist_ok=True)
                print(f"\n✅ Created session for batch {batch_num}: {os.path.basename(new_path)}")
                print(f"   Will process rows {start_row}-{start_row + BATCH_SIZE - 1}")
                return new_path, start_row, True
            except ValueError:
                print("❌ Invalid batch number")
                continue

        elif choice.isdigit():
            idx = int(choice) - 1
            if 0 <= idx < len(sessions_data):
                selected = sessions_data[idx]
                print(f"\n✅ Selected: {selected['name']}")
                print(f"   Status: {selected['status']}")

                # Determine start row
                if selected['last_row'] is not None:
                    if selected['batch_end'] and selected['last_row'] >= selected['batch_end'] - 1:
                        # Batch complete - ask what to do
                        print("\n⚠️ This batch appears complete!")
                        next_action = input("Start NEXT batch (y) or re-check last rows (n)? (y/n): ").strip().lower()
                        if next_action == 'y':
                            start_row = selected['batch_end']
                            # Create new session for next batch
                            timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                            batch_num = (selected['batch_end'] // BATCH_SIZE) + 1
                            new_path = os.path.join(base_dir, f"structured_session_{timestamp}_batch{batch_num}")
                            os.makedirs(new_path, exist_ok=True)
                            print(f"✅ Created new session for batch {batch_num}")
                            return new_path, start_row, True
                        else:
                            start_row = max(0, selected['last_row'] - 10)  # Re-check last 10 rows
                    else:
                        start_row = selected['last_row'] + 1
                else:
                    start_row = selected['batch_start'] if selected['batch_start'] is not None else 0

                print(f"   Will resume from row: {start_row}")
                return selected['path'], start_row, False
            else:
                print("❌ Invalid selection number")
                continue
        else:
            print("❌ Invalid choice. Please try again.")

# === MAIN SESSION SELECTION ===
GDRIVE_DIR, START_ROW, is_new_session = select_or_create_session()

print("\n" + "=" * 70)
print("📁 Working directory:", os.path.basename(GDRIVE_DIR))
print("🎯 Starting from row:", START_ROW)
print("🆕 New session:", "Yes" if is_new_session else "No")
print("=" * 70)

# Save session metadata
session_info_file = os.path.join(GDRIVE_DIR, "session_info.txt")
with open(session_info_file, 'a') as f:
    if is_new_session:
        f.write(f"Session created: {datetime.datetime.now().isoformat()}\n")
        f.write(f"Initial start row: {START_ROW}\n")
        f.write(f"Batch size: {BATCH_SIZE}\n")
    else:
        f.write(f"\nResumed: {datetime.datetime.now().isoformat()}\n")
        f.write(f"Resumed from row: {START_ROW}\n")

# === YOUR ORIGINAL PROCESSING FUNCTIONS ===

def parse_xml_to_flat_json(xml_response):
    """
    Parse XML response to flat JSON with no nesting.
    Creates clean columns for statistical analysis.
    """
    try:
        # Clean the response
        xml_response = xml_response.strip()

        # If it's not XML, try to extract XML from it
        if not xml_response.startswith('<'):
            # Look for XML content in the response
            xml_match = re.search(r'<analysis>.*</analysis>', xml_response, re.DOTALL)
            if xml_match:
                xml_response = xml_match.group(0)
            else:
                return {"error": "No XML found in response", "raw_response": xml_response}

        # Parse XML
        root = ET.fromstring(xml_response)

        # Initialize flat dictionary
        flat_data = {}

        # Extract variables section
        variables = root.find('variables')
        if variables is not None:
            for child in variables:
                tag_name = child.tag
                # Remove 'var_' prefix if present for cleaner column names
                clean_name = tag_name.replace('var_', '')
                flat_data[clean_name] = child.text if child.text else "none"

        return flat_data

    except Exception as e:
        return {
            "error": f"XML parsing error: {str(e)}",
            "raw_response": xml_response[:500]
        }

def construct_structured_prompt(entry_text):
    """Construct prompt with the structured codebook."""
    return f"""{STRUCTURED_CODEBOOK}

IMPORTANT REMINDERS:
- Answer EVERY variable listed above
- Use "none" if information is not present
- Base answers ONLY on this text (do not infer)
- Provide both coded value and descriptive detail for each variable
- Follow the exact XML format specified

EXCERPT TO ANALYZE:
{entry_text}

Provide your complete XML response:"""

def load_data_file(filepath):
    """Try multiple methods to load the data file."""

    # Check if file exists in current directory first
    if not os.path.exists(filepath):
        # Try common alternative locations
        alternatives = [
            f"/content/{filepath}",
            f"/content/drive/MyDrive/{filepath}",
            filepath.replace('.xlsx', '.csv'),
            filepath.replace('.xlsx', '.xls')
        ]

        for alt in alternatives:
            if os.path.exists(alt):
                filepath = alt
                print(f"📁 Found file at: {filepath}")
                break

    # Try loading with different methods
    try:
        df = pd.read_excel(filepath, engine="openpyxl")
        print(f"✅ Loaded as .xlsx: {len(df)} rows")
        return df
    except:
        pass

    try:
        df = pd.read_excel(filepath)
        print(f"✅ Loaded with default engine: {len(df)} rows")
        return df
    except:
        pass

    try:
        df = pd.read_csv(filepath)
        print(f"✅ Loaded as CSV: {len(df)} rows")
        return df
    except:
        pass

    print(f"❌ Could not load file: {filepath}")
    return None

def calculate_cost(input_tokens, output_tokens):
    """Calculate API cost based on token usage."""
    # Claude 3.5 Sonnet pricing (as of 2024)
    input_rate = 0.003 / 1000  # $3 per million input tokens
    output_rate = 0.015 / 1000  # $15 per million output tokens
    return (input_tokens * input_rate) + (output_tokens * output_rate)

def save_intermediate_results(current_row, responses, input_tokens, output_tokens, batch_start, df):
    """Save intermediate results with clean structure."""

    print(f"\n💾 SAVING INTERMEDIATE RESULTS (Row {current_row})")

    # Save current index
    with open(os.path.join(GDRIVE_DIR, 'last_index.txt'), 'w') as f:
        f.write(str(current_row))

    # Create results dataframe
    rows_processed = current_row - batch_start + 1
    df_partial = df.iloc[batch_start:current_row + 1].copy()

    # Add response columns
    if responses and len(responses) >= rows_processed:
        batch_responses = responses[:rows_processed]
        valid_responses = [r for r in batch_responses if r is not None and isinstance(r, dict)]

        if valid_responses:
            # Get all unique keys
            all_keys = set()
            for resp in valid_responses:
                all_keys.update(resp.keys())

            # Add columns
            for key in sorted(all_keys):
                col_values = []
                for resp in batch_responses:
                    if resp and isinstance(resp, dict):
                        col_values.append(resp.get(key))
                    else:
                        col_values.append(None)
                df_partial[key] = col_values

    # Save file
    batch_num = (batch_start // BATCH_SIZE) + 1
    filename = f"batch_{batch_num}_intermediate_{batch_start}_{current_row}.xlsx"
    filepath = os.path.join(GDRIVE_DIR, filename)
    df_partial.to_excel(filepath, index=False)

    # Save cost summary
    cost = calculate_cost(input_tokens, output_tokens)
    cost_file = os.path.join(GDRIVE_DIR, f'batch_{batch_num}_cost_intermediate.txt')
    with open(cost_file, 'w') as f:
        f.write(f"Batch {batch_num} - Intermediate\n")
        f.write(f"Rows: {batch_start} to {current_row}\n")
        f.write(f"Processed: {rows_processed}\n")
        f.write(f"Cost: ${cost:.4f}\n")
        f.write(f"Updated: {datetime.datetime.now().isoformat()}\n")

    print(f"✅ Saved: {filename}")
    print(f"💰 Cost so far: ${cost:.4f}")

def process_entries_structured(df):
    """
    Main processing function with proper batch management.
    """
    global START_ROW

    total_input_tokens = 0
    total_output_tokens = 0
    responses = []

    # Determine batch boundaries
    batch_info_file = os.path.join(GDRIVE_DIR, 'batch_info.txt')

    if os.path.exists(batch_info_file) and not is_new_session:
        # Resume existing batch
        with open(batch_info_file, 'r') as f:
            lines = f.readlines()
            if len(lines) >= 2:
                batch_start = int(lines[0].strip())
                batch_end = int(lines[1].strip())
                print(f"📊 Resuming batch: {batch_start}-{batch_end-1}")
    else:
        # New batch
        batch_start = START_ROW
        batch_end = min(batch_start + BATCH_SIZE, len(df))

        # Save batch info
        with open(batch_info_file, 'w') as f:
            f.write(f"{batch_start}\n{batch_end}\n")

        print(f"📊 Starting new batch: {batch_start}-{batch_end-1}")

    # Initialize responses for skipped rows
    for i in range(batch_start, START_ROW):
        responses.append(None)

    print(f"\n🚀 PROCESSING")
    print(f"Model: {MODEL_NAME}")
    print(f"Temperature: {TEMPERATURE}")
    print(f"Processing rows: {START_ROW} to {batch_end-1}")
    print(f"Total rows in batch: {batch_end - batch_start}")

    log_path = os.path.join(GDRIVE_DIR, LOG_FILE)

    # Create Anthropic client
    client = Anthropic(api_key=ANTHROPIC_API_KEY)

    with open(log_path, "a", encoding="utf-8") as log:
        log.write(f"\n=== Processing started at {datetime.datetime.now().isoformat()} ===\n")
        log.write(f"Batch: {batch_start}-{batch_end-1}, starting from row {START_ROW}\n")

        for i in range(START_ROW, batch_end):
            try:
                row = df.iloc[i]
                entry_text = row.get(TEXT_COLUMN)

                # Check for empty text
                if pd.isna(entry_text) or entry_text is None:
                    responses.append(None)
                    print(f"⏭️ Row {i}: Empty text, skipping")
                    continue

                entry_text = str(entry_text).strip()

                if not entry_text or entry_text == 'nan':
                    responses.append(None)
                    print(f"⏭️ Row {i}: Empty text, skipping")
                    continue

                # Construct prompt
                prompt = construct_structured_prompt(entry_text)

                print(f"\n🔍 Processing row {i}/{batch_end-1}")
                print(f"   Batch progress: {((i - batch_start + 1) / (batch_end - batch_start)) * 100:.1f}%")
                print(f"   Text preview: {entry_text[:100]}...")

                # Call Claude API
                response = client.messages.create(
                    model=MODEL_NAME,
                    max_tokens=3000,
                    temperature=TEMPERATURE,
                    messages=[{"role": "user", "content": prompt}]
                )

                total_input_tokens += response.usage.input_tokens
                total_output_tokens += response.usage.output_tokens
                response_text = response.content[0].text.strip()

                # Parse XML to flat JSON
                parsed_data = parse_xml_to_flat_json(response_text)

                # Add raw response for reference
                parsed_data['raw_llm_response'] = response_text[:1000]

                responses.append(parsed_data)

                # Count successful fields
                field_count = len([k for k in parsed_data.keys() if not k.startswith('error')])
                print(f"✅ Row {i}: Extracted {field_count} fields")

                timestamp = datetime.datetime.now().isoformat()
                log.write(f"[{timestamp}] Row {i}: Success - {field_count} fields\n")

                # Save progress
                with open(os.path.join(GDRIVE_DIR, 'last_index.txt'), 'w') as f:
                    f.write(str(i))

                # Auto-save
                if (i - batch_start + 1) % SAVE_EVERY == 0:
                    save_intermediate_results(i, responses, total_input_tokens,
                                            total_output_tokens, batch_start, df)

                # Cost tracking
                if (i - batch_start) % 10 == 0:
                    current_cost = calculate_cost(total_input_tokens, total_output_tokens)
                    print(f"💰 Current cost: ${current_cost:.4f}")

            except Exception as e:
                timestamp = datetime.datetime.now().isoformat()
                error_msg = f"[{timestamp}] Row {i}: Error - {str(e)}\n"
                log.write(error_msg)
                print(f"❌ {error_msg.strip()}")
                responses.append({"error": str(e), "row": i})

        log.write(f"=== Processing ended at {datetime.datetime.now().isoformat()} ===\n")

    return responses, total_input_tokens, total_output_tokens, batch_start, batch_end

# === MAIN EXECUTION ===
print("\n📁 Loading data file...")
df = load_data_file(EXCEL_PATH)

if df is None:
    print("❌ Failed to load data file. Please check the file path and format.")
    sys.exit(1)

print(f"✅ Data loaded: {len(df)} rows, {len(df.columns)} columns")

# Check for text column
if TEXT_COLUMN not in df.columns:
    print(f"⚠️ Column '{TEXT_COLUMN}' not found. Available columns:")
    for col in df.columns:
        print(f"   - {col}")

    # Try to find a text column
    for col in df.columns:
        if 'text' in col.lower() or 'content' in col.lower():
            TEXT_COLUMN = col
            print(f"📝 Using detected column: {TEXT_COLUMN}")
            break
    else:
        print("❌ No suitable text column found!")
        sys.exit(1)

# Display sample
non_null = df[TEXT_COLUMN].dropna()
if len(non_null) > 0:
    print(f"\n📝 Sample text: {str(non_null.iloc[0])[:150]}...")

# Process entries
print(f"\n🚀 Starting processing...")
responses, total_input_tokens, total_output_tokens, batch_start, batch_end = process_entries_structured(df)

# === SAVE FINAL RESULTS ===
print(f"\n🎉 BATCH PROCESSING COMPLETE!")
print("=" * 50)

if responses:
    # Create final dataframe
    batch_df = df.iloc[batch_start:batch_end].copy()

    # Add response columns
    valid_responses = [r for r in responses if r is not None and isinstance(r, dict)]

    if valid_responses:
        # Get all unique keys
        all_keys = set()
        for resp in valid_responses:
            all_keys.update(resp.keys())

        # Add columns
        for key in sorted(all_keys):
            col_values = []
            for resp in responses:
                if resp and isinstance(resp, dict):
                    col_values.append(resp.get(key))
                else:
                    col_values.append(None)
            batch_df[key] = col_values

    # Save final batch
    batch_num = (batch_start // BATCH_SIZE) + 1
    final_path = os.path.join(GDRIVE_DIR, f"batch_{batch_num}_final_{batch_start}_{batch_end-1}.xlsx")
    batch_df.to_excel(final_path, index=False)

    # Create summary
    final_cost = calculate_cost(total_input_tokens, total_output_tokens)
    total_batches = (len(df) // BATCH_SIZE) + (1 if len(df) % BATCH_SIZE else 0)

    summary = f"""
STRUCTURED BEAD CODING - BATCH {batch_num}/{total_batches} COMPLETE
============================================================

📊 BATCH SUMMARY:
- Batch number: {batch_num} of {total_batches}
- Rows processed: {batch_start} to {batch_end-1}
- Total entries: {batch_end - batch_start}
- Successful: {len([r for r in responses if r and not r.get('error')])}
- Input tokens: {total_input_tokens:,}
- Output tokens: {total_output_tokens:,}
- Batch cost: ${final_cost:.4f}

📊 OVERALL PROGRESS:
- Total dataset: {len(df)} rows
- Completed: {batch_end} rows ({(batch_end/len(df))*100:.1f}%)
- Remaining: {len(df) - batch_end} rows

📁 OUTPUT FILES:
- Final results: {os.path.basename(final_path)}
- Session folder: {os.path.basename(GDRIVE_DIR)}

Generated: {datetime.datetime.now().isoformat()}
"""

    print(summary)

    # Save summary
    with open(os.path.join(GDRIVE_DIR, f'batch_{batch_num}_summary.txt'), 'w') as f:
        f.write(summary)

    print(f"✅ Results saved to: {GDRIVE_DIR}")

    if batch_end < len(df):
        print(f"\n⏭️ TO CONTINUE:")
        print(f"Run this script again and it will offer to continue from row {batch_end}")
else:
    print("\n⚠️ No entries were processed.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔬 STRUCTURED BEAD TRADE CODING - V3 WITH SMART SESSIONS
📅 Current date: 2025-09-05 08:02:09

📁 EXISTING SESSIONS:

1. structured_session_2025-09-04_09-07-11
   📅 Created: 2025-09-04 09-07-11
   📊 Batch: rows 0-1999
   ✏️ Last row: 1999
   📁 Files: 41 (51.1 MB)
   📈 Status: complete

2. structured_session_2025-09-02_09-21-53
   📅 Created: 2025-09-02 09-21-53
   📊 Batch: rows 1900-3899
   ✏️ Last row: 3899
   📁 Files: 79 (107.6 MB)
   📈 Status: complete

3. structured_session_2025-09-02_09-07-26
   📅 Created: 2025-09-02 09-07-26
   📁 Files: 0 (0.0 MB)
   📈 Status: not started

4. structured_session_2025-09-02_09-04-11
   📅 Created: 2025-09-02 09-04-11
   📁 Files: 0 (0.0 MB)
   📈 Status: not started

5. structured_session_2025-09-02_09-03-06
   📅 Created: 2025-09-02 09-03-06
   📁 Files: 0 (0.0 MB)
   📈 Status: not started


OPTIONS:
  1-10 : Resume specific sess

/tmp/ipython-input-3998825229.py:687: DeprecationWarning: The model 'claude-3-5-sonnet-20241022' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


✅ Row 0: Extracted 29 fields
💰 Current cost: $0.0218

🔍 Processing row 1/1999
   Batch progress: 0.1%
   Text preview: KINGDOM OF MARR VECOS Fo.4. They wear woolen cloaks called cur?ias corcas that reach up to their elb...
✅ Row 1: Extracted 28 fields
⏭️ Row 2: Empty text, skipping

🔍 Processing row 3/1999
   Batch progress: 0.2%
   Text preview: WANDERINGS IN BIBLE LANDS. 271 Before reaching the first cataract the scenery along the Nile changes...
✅ Row 3: Extracted 29 fields

🔍 Processing row 4/1999
   Batch progress: 0.2%
   Text preview: BOOK FOUR OF the king's all the penalties, ailments, where the warden resides together and slanders,...
✅ Row 4: Extracted 29 fields

🔍 Processing row 5/1999
   Batch progress: 0.3%
   Text preview: WANDERINGS IN BIBLE LANDS. 273 conducts himself in accordance with the standard rules of society he ...
✅ Row 5: Extracted 48 fields

🔍 Processing row 6/1999
   Batch progress: 0.4%
   Text preview: *% 280 WANDERINGS IN BIBLE LANDS. Nile. Cut off from t

/tmp/ipython-input-3998825229.py:687: DeprecationWarning: The model 'claude-3-5-sonnet-20241022' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


✅ Row 50: Extracted 29 fields
💰 Current cost: $1.0045

🔍 Processing row 51/1999
   Batch progress: 2.6%
   Text preview: Bulgarians by origin. It is in the only tavern in the village that the Mayor, an old peasant, receiv...
✅ Row 51: Extracted 29 fields

🔍 Processing row 52/1999
   Batch progress: 2.6%
   Text preview: 40 The "Lion-hearted" Bishop. Buganda by this road, learned from the head-man of a caravan that the ...
✅ Row 52: Extracted 49 fields

🔍 Processing row 53/1999
   Batch progress: 2.7%
   Text preview: WHERE "THE MAN WITH THE SPECTACLES" LIVES MISSION-HOUSE AT BUKASA. THE SESE AND OTHER ISLANDS. OW le...
✅ Row 53: Extracted 28 fields

🔍 Processing row 54/1999
   Batch progress: 2.8%
   Text preview: JOURNAL OF AN OFFicer, &c. 11 civilised in their manufacture than the habitations; several of the na...
✅ Row 54: Extracted 29 fields

🔍 Processing row 55/1999
   Batch progress: 2.8%
   Text preview: Lazy Animals. 65 plentiful supply of fine grass on our bridge, over this ou

...

✅ Row 1850: Extracted 48 fields
💰 Current cost: $43.0387

🔍 Processing row 1851/1999
   Batch progress: 92.6%
   Text preview: 146 ANGOLA AND THE PJVER CONGO, [ch. v. allowed to plunder and carry off prisoners as slaves. These ...
✅ Row 1851: Extracted 28 fields

🔍 Processing row 1852/1999
   Batch progress: 92.7%
   Text preview: Plate XII. Maxilla, and Barber's Shop â€”Canning Corpse to Lmi il -Qui.sima W uiuen, and manner of p...
✅ Row 1852: Extracted 49 fields

🔍 Processing row 1853/1999
   Batch progress: 92.7%
   Text preview: CH. Yi.] CUBIOUS CUSTOM, 167 white bar across its tail, and making its nest of the hair of different...
✅ Row 1853: Extracted 28 fields

🔍 Processing row 1854/1999
   Batch progress: 92.8%
   Text preview: 168 ANGOLA AND TEE BIVER CONGO, [ch. vi. this absurd idea entertained exactly in the same manner in ...
✅ Row 1854: Extracted 29 fields

🔍 Processing row 1855/1999
   Batch progress: 92.8%
   Text preview: CH. VI.] EGITO. 169 and becomes a filthy mass of 

/tmp/ipython-input-3998825229.py:687: DeprecationWarning: The model 'claude-3-5-sonnet-20241022' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


✅ Row 1900: Extracted 29 fields
💰 Current cost: $44.2094

🔍 Processing row 1901/1999
   Batch progress: 95.1%
   Text preview: 60 Aprilâ€™ 19 . Caution to the Commandant. When the robbers are driven out of their dens and lurkin...
✅ Row 1901: Extracted 29 fields

🔍 Processing row 1902/1999
   Batch progress: 95.2%
   Text preview: 69 known in all ages, and, although when hard pressed 1777, Ju "â€˜ J J ' they defend themselves in ...
✅ Row 1902: Extracted 49 fields

🔍 Processing row 1903/1999
   Batch progress: 95.2%
   Text preview: 70 1775, Some of his men, and lie himself, were wounded with January 13. poisoned arrows, but no liv...
✅ Row 1903: Extracted 26 fields

🔍 Processing row 1904/1999
   Batch progress: 95.2%
   Text preview: 71 Kaffers, only got one, who had been a year among 1777, December 4. them ; a KafTer returned with ...
✅ Row 1904: Extracted 29 fields

🔍 Processing row 1905/1999
   Batch progress: 95.3%
   Text preview: 102 Commentary Whether the Kaffer be descended fr

/tmp/ipython-input-3998825229.py:687: DeprecationWarning: The model 'claude-3-5-sonnet-20241022' is deprecated and will reach end-of-life on October 22, 2025.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  response = client.messages.create(


✅ Row 1950: Extracted 29 fields
💰 Current cost: $45.3494

🔍 Processing row 1951/1999
   Batch progress: 97.6%
   Text preview: 211 a saw out of a luyk ; a trip was made to the wreck, The Kaffers. 1687, and various articles were...
✅ Row 1951: Extracted 20 fields

🔍 Processing row 1952/1999
   Batch progress: 97.7%
   Text preview: 212 The Kaffers. them, who had a short time before lost their ship arid 1687, March. all their prope...
✅ Row 1952: Extracted 26 fields

🔍 Processing row 1953/1999
   Batch progress: 97.7%
   Text preview: 213 lain bean, in size and taste not unlike the European The Kaffers. horse bean ; also some roots, ...
✅ Row 1953: Extracted 29 fields

🔍 Processing row 1954/1999
   Batch progress: 97.8%
   Text preview: 214 The Kaffers. salted and smoked beef, pumpkins, and water, all 1687, March. purchased from the na...
✅ Row 1954: Extracted 20 fields

🔍 Processing row 1955/1999
   Batch progress: 97.8%
   Text preview: 215 they appear to have reared in such numbers, b